DOC:
https://rawg.io/apidocs
- Count: 345727
- Time taken: 373,744.71608‬ s = 103 hrs = 4 s


- https://api.rawg.io/docs/
- https://api.rawg.io/api/games


## Get the list of all games with its number

In [1]:
import json
import requests
from pprint import pprint
import os
from time import time
import concurrent.futures
import functools
import os

In [6]:
headers = { 'User-Agent': 'App Name: Education purpose',}
def request_and_cache(url, exclude, N):
    # Run through N pages
    for i in range(N):
        print(url)
        D = {}
        json_data = json.loads(requests.get(url, headers=headers).text)
        
        # Get wanted data
        for game in json_data["results"]:
            D[game["id"]] = {k:v for k,v in game.items() if k not in exclude}
            
            # Special handler
            D[game["id"]]["platforms"] = []
            for data in game["platforms"]:
                D[game["id"]]["platforms"].append(data["platform"]["name"])
                
            D[game["id"]]["genres"] = []
            for data in game["genres"]:
                D[game["id"]]["genres"].append(data["platform"]["name"])
            
        # Save data
        page_no = int(url.split("page=")[-1])
        json.dump(D, open(fr"data/{page_no}.json", "w", encoding="utf8"))
        if not json_data['next']: break
        url = json_data['next']
    print(f"Done from {page_no - N} to {page_no}")
        

Time reduce from 4 hours to 40 minutes

In [19]:
exclude = ["saturated_color", "dominant_color", "parent_platforms", "stores", "clip", "short_screenshots"]

max_workers = 86
step = int(17286/max_workers)
start_number = range(1, 17286, step)
url = "https://api.rawg.io/api/games?page=1"
urls = [url[:-1] + str(i) for i in start_number]
t0=time()
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    temp = functools.partial(request_and_cache, exclude=exclude, N=step)
    executor.map(temp, urls)
print(time()-t0)

Specific game:
- https://api.rawg.io/api/games/rimworld
- https://api.rawg.io/api/games/grand-theft-auto-v
- https://rawg.io/games/grand-theft-auto-v

## Get info of each game

In [1]:
import os
import json
import csv
with open("data/game_id.csv", "w") as f:
    csv_file = csv.writer(f, lineterminator="\n")
    for file in os.listdir("rawdata/"):
        try:
            json_data = json.load(open(f"rawdata/{file}", "r"))
        except:
            print(file)
        for game in json_data.values():
            csv_file.writerow([game["id"], game["slug"]])
        visited.add(file)

203.json


In [2]:
urls = ("https://api.rawg.io/api/games/grand-theft-auto-v", "https://api.rawg.io/api/games/rimworld")

headers = { 'User-Agent': 'App Name: Education purpose',}

include = {"id",
           "slug",
           "name",
           "metacritic",
           "released",
           "tba",
           "rating",
           "rating_top",
#            "ratings",
           "added",
           "added_by_status",
           "playtime",
           "reddit_count",
           "twitch_count",
           "youtube_count",
           "ratings_count",
           "suggestions_count",
           "game_series_count",
           "reviews_count",
           "platforms",
           "developers",
           "genres",
           "tags",
           "publishers",
           "esrb_rating",
           }
for url in urls:
    D = {}
    json_data = json.loads(requests.get(url, headers=headers).text)
    
    D = {k:v for k,v in json_data.items() if k in include}
    
    D["platforms"] = []
    for platform in json_data["platforms"]:
        D["platforms"].append(platform["platform"]["name"])
    
    
    for key in ("developers", "genres", "tags", "publishers"):
        D[key] = []
        for data in json_data[key]:
            D[key].append(data["name"])
    
    name = D["id"]
    json.dump(D, open(f"data/{name}.json","w", encoding="utf-8"))
